## Avinash Kappa - Assignment 4b 

###Trials and observations:

**In my first submission last week, I got test accuracy of 85.84%.  **   

After having tried a few things, what worked to get >85% accuracy was reducing the batch size to 64 and increasing filters to 24.  
###To reach 87% accuracy, these are the things I tried :  

I tried for different dropouts like 0.5 and 0.3 which didnt give better results so I left the dropout as 0.2.  

I trained the same model with 85.84% accuracy again to see if it might give any better results. It didn't.  

I decreased the batch size to 32 and increased filters to 32. Each epoch was taking too long for Colab and the runtime was dying intermittently.   

I then used data augmentation along with two convolution layers that are added before the 1st convolution layer.  

These 2 convolution layers are of filter dimensions 1 X 7 and 7 X 1.  

In the last trial on Colab, the runtime stopped when the training was at epoch 13. Validation accuracy at this point was 77.7%  

I took the saved weights and resumed the training with epochs = 38.  

**At epoch 25/38, which is the 37th epoch overall out of 50, the validation accuracy reached 87.4%**    

**The training ended with the best validation accuracy of 88.02.  **

Using the trained model as it is to evaluate, test accuracy of 87.3 was observed.  

Upon loading the weights of best validation accuracy of 88.02%, the test accuracy observed was 88.03%.

Logs are attached for the two runs, the first one that lasted for 12 epochs and the other that went on for rest of the 38 epochs.  




In [1]:
# https://keras.io/
!pip install keras
import keras

Using TensorFlow backend.


In [2]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
import tensorflow as tf
from keras import backend as k
from keras.callbacks import ModelCheckpoint,CSVLogger

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 64
num_classes = 10
epochs = 50
l = 12
num_filter = 24
compression = 0.5
dropout_rate = 0

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:

input = Input(shape=(img_height, img_width, channel,))
Conv2D_7x1 = Conv2D(num_filter,(7,1),use_bias=False,padding='same')(input)
Conv2D_1x7 = Conv2D(num_filter,(1,7),use_bias=False,padding='same')(Conv2D_7x1)
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(Conv2D_1x7)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


## Saving the weights during the training

In [0]:
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
trainlog = CSVLogger("training.log")
callbacks_list = [checkpoint,trainlog]

In [12]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 24)   504         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 32, 32, 24)   4032        conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 32, 32, 24)   5184        conv2d_2[0][0]                   
__________________________________________________________________________________________________
batch_norm

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])


## Adding data augmentation

In [0]:
from keras.preprocessing.image import ImageDataGenerator

train_data_gen = ImageDataGenerator( 
                          rotation_range=10, 
                          width_shift_range=0.2, 
                          height_shift_range=0.2, 
                          zoom_range=0.25,
                          horizontal_flip=True)

validation_data_gen = ImageDataGenerator()
train_generator = train_data_gen.flow(x_train, y_train, batch_size=batch_size)
validation_generator = validation_data_gen.flow(x_test, y_test, batch_size=batch_size)

In [0]:
model.fit_generator(train_generator, 
                    steps_per_epoch=train_generator.n//batch_size, 
                    epochs=50, 
                    callbacks=callbacks_list, 
                    verbose =1,
                    validation_data=validation_generator, 
                    validation_steps=validation_generator.n//batch_size)


Epoch 1/50
781/781 [==============================] - 313s 401ms/step - loss: 1.7094 - acc: 0.3701 - val_loss: 1.6283 - val_acc: 0.4266

Epoch 00001: val_acc improved from -inf to 0.42658, saving model to weights.best.hdf5
Epoch 2/50
 88/781 [==>...........................] - ETA: 4:12 - loss: 1.5060 - acc: 0.4547

781/781 [==============================] - 300s 385ms/step - loss: 1.3660 - acc: 0.5082 - val_loss: 1.1774 - val_acc: 0.5698

Epoch 00002: val_acc improved from 0.42658 to 0.56981, saving model to weights.best.hdf5
Epoch 3/50
141/781 [====>.........................] - ETA: 3:52 - loss: 1.2312 - acc: 0.5541

781/781 [==============================] - 301s 386ms/step - loss: 1.1506 - acc: 0.5892 - val_loss: 1.1494 - val_acc: 0.6010

Epoch 00003: val_acc improved from 0.56981 to 0.60096, saving model to weights.best.hdf5
Epoch 4/50
155/781 [====>.........................] - ETA: 3:48 - loss: 1.0584 - acc: 0.6218

781/781 [==============================] - 301s 385ms/step - loss: 1.0159 - acc: 0.6407 - val_loss: 1.2112 - val_acc: 0.6072

Epoch 00004: val_acc improved from 0.60096 to 0.60717, saving model to weights.best.hdf5
Epoch 5/50
159/781 [=====>........................] - ETA: 3:45 - loss: 0.9632 - acc: 0.6567

781/781 [==============================] - 302s 387ms/step - loss: 0.9251 - acc: 0.6738 - val_loss: 0.9075 - val_acc: 0.6865

Epoch 00005: val_acc improved from 0.60717 to 0.68650, saving model to weights.best.hdf5
Epoch 6/50
160/781 [=====>........................] - ETA: 3:46 - loss: 0.8601 - acc: 0.6971

...

781/781 [==============================] - 300s 384ms/step - loss: 0.6981 - acc: 0.7552 - val_loss: 0.8115 - val_acc: 0.7276

Epoch 00009: val_acc improved from 0.70212 to 0.72756, saving model to weights.best.hdf5
Epoch 10/50
162/781 [=====>........................] - ETA: 3:44 - loss: 0.6706 - acc: 0.7686

781/781 [==============================] - 301s 385ms/step - loss: 0.6657 - acc: 0.7674 - val_loss: 1.1016 - val_acc: 0.6695

Epoch 00010: val_acc did not improve from 0.72756
Epoch 11/50
184/781 [======>.......................] - ETA: 3:36 - loss: 0.6534 - acc: 0.7751

781/781 [==============================] - 300s 384ms/step - loss: 0.6445 - acc: 0.7764 - val_loss: 0.9273 - val_acc: 0.7095

Epoch 00011: val_acc did not improve from 0.72756
Epoch 12/50
191/781 [======>.......................] - ETA: 3:33 - loss: 0.6054 - acc: 0.7917

781/781 [==============================] - 303s 388ms/step - loss: 0.6157 - acc: 0.7846 - val_loss: 0.6648 - val_acc: 0.7778

Epoch 00012: val_acc improved from 0.72756 to 0.77784, saving model to weights.best.hdf5
Epoch 13/50
168/781 [=====>........................] - ETA: 3:46 - loss: 0.6059 - acc: 0.7880

378/781 [=============>................] - ETA: 2:28 - loss: 0.5937 - acc: 0.7923

### Training Log for Epoch 1 to 12  
#### epoch,acc,loss,val_acc,val_loss  
  

0,0.3701537968599808,1.709243738777322,0.42658253205128205,1.6283374016101544
1,0.5080903556552387,1.3661691263599756,0.5698116987179487,1.177447960162774
2,0.5891340916372957,1.1504799168830269,0.6009615384615384,1.1494317127343936
3,0.6407000961230375,1.0158001427007233,0.6071714743589743,1.2112317566688244
4,0.6736622877282922,0.9254216284218678,0.6864983974358975,0.9074608779106385
5,0.7051826337712271,0.8393835356433359,0.6469350961538461,1.0532327664968295
6,0.7236462672220442,0.7880550019445423,0.5992588141025641,1.440164443009939
7,0.7409083627042614,0.7425034802069231,0.7021233974358975,0.8715929431028855
8,0.7550865107337392,0.6983746800694624,0.7275641025641025,0.8114863913028668
9,0.7675024030759372,0.6656979080271698,0.6694711538461539,1.1016396081600435
10,0.7764138096763857,0.6441716925527223,0.7095352564102564,0.9272776737045019
**11,0.7846443447612944,0.6156081741568905,0.7778445512820513,0.6647945105647429**

###Training was interrupted due to runtime termination after epoch 12. Using saved weights to resume training for another 38 epochs

In [15]:
!ls

datalab  training.log  weights.best.hdf5


In [0]:
from keras.models import load_model
model = load_model("weights.best.hdf5")

In [19]:
model.fit_generator(train_generator, 
                    steps_per_epoch=train_generator.n//batch_size, 
                    epochs=38, 
                    callbacks=callbacks_list, 
                    verbose =1,
                    validation_data=validation_generator, 
                    validation_steps=validation_generator.n//batch_size)

Epoch 1/38
781/781 [==============================] - 310s 397ms/step - loss: 0.5094 - acc: 0.8241 - val_loss: 0.7401 - val_acc: 0.7664

Epoch 00001: val_acc improved from -inf to 0.76643, saving model to weights.best.hdf5
Epoch 2/38
 88/781 [==>...........................] - ETA: 4:14 - loss: 0.4705 - acc: 0.8386

781/781 [==============================] - 304s 389ms/step - loss: 0.4986 - acc: 0.8278 - val_loss: 0.6024 - val_acc: 0.7916

Epoch 00002: val_acc improved from 0.76643 to 0.79157, saving model to weights.best.hdf5
Epoch 3/38
141/781 [====>.........................] - ETA: 3:55 - loss: 0.4932 - acc: 0.8267

781/781 [==============================] - 304s 389ms/step - loss: 0.4857 - acc: 0.8293 - val_loss: 0.6767 - val_acc: 0.7825

Epoch 00003: val_acc did not improve from 0.79157
Epoch 4/38
179/781 [=====>........................] - ETA: 3:41 - loss: 0.4606 - acc: 0.8413

781/781 [==============================] - 303s 388ms/step - loss: 0.4744 - acc: 0.8339 - val_loss: 0.6572 - val_acc: 0.7882

Epoch 00004: val_acc did not improve from 0.79157
Epoch 5/38
190/781 [======>.......................] - ETA: 3:37 - loss: 0.4588 - acc: 0.8431

781/781 [==============================] - 303s 388ms/step - loss: 0.4634 - acc: 0.8389 - val_loss: 0.6838 - val_acc: 0.7889

Epoch 00005: val_acc did not improve from 0.79157
Epoch 6/38
194/781 [======>.......................] - ETA: 3:34 - loss: 0.4657 - acc: 0.8392

...

781/781 [==============================] - 302s 387ms/step - loss: 0.3080 - acc: 0.8908 - val_loss: 0.5773 - val_acc: 0.8284

Epoch 00034: val_acc did not improve from 0.87440
Epoch 35/38
194/781 [======>.......................] - ETA: 3:34 - loss: 0.2951 - acc: 0.8963

781/781 [==============================] - 303s 388ms/step - loss: 0.2995 - acc: 0.8955 - val_loss: 0.3758 - val_acc: 0.8802

Epoch 00035: val_acc improved from 0.87440 to 0.88021, saving model to weights.best.hdf5
Epoch 36/38
169/781 [=====>........................] - ETA: 3:43 - loss: 0.3051 - acc: 0.8940

781/781 [==============================] - 303s 388ms/step - loss: 0.3027 - acc: 0.8937 - val_loss: 0.4311 - val_acc: 0.8656

Epoch 00036: val_acc did not improve from 0.88021
Epoch 37/38
186/781 [======>.......................] - ETA: 3:38 - loss: 0.3000 - acc: 0.8939

781/781 [==============================] - 303s 388ms/step - loss: 0.3022 - acc: 0.8951 - val_loss: 0.4354 - val_acc: 0.8632

Epoch 00037: val_acc did not improve from 0.88021
Epoch 38/38
192/781 [======>.......................] - ETA: 3:35 - loss: 0.2965 - acc: 0.8971

781/781 [==============================] - 303s 388ms/step - loss: 0.2965 - acc: 0.8962 - val_loss: 0.4048 - val_acc: 0.8728

Epoch 00038: val_acc did not improve from 0.88021


### Training Log for Epoch 13 to 50   

#### epoch,acc,loss,val_acc,val_loss    


0,0.8242550464594681,0.5087492304213419,0.766426282051282,0.7401369857864503
1,0.8277995834668376,0.49855676676286637,0.7915665064102564,0.6024200304960593
2,0.8292614546619673,0.4857004265474303,0.7824519230769231,0.6766831647509184
3,0.8340676065363666,0.4739943714869095,0.7881610576923077,0.6571524939857997
4,0.8388937840435757,0.46314855662342524,0.7888621794871795,0.683842563476318
5,0.8409564242230054,0.4573263793333873,0.7519030448717948,0.8514463431560076
6,0.8462031400192246,0.44579366845452073,0.8267227564102564,0.5136595924313252
7,0.849927907721884,0.43644575874456343,0.8055889423076923,0.5981472298885003
8,0.8518904197372636,0.4282795712184692,0.8422475961538461,0.486338974096072
9,0.8518904197372636,0.4228127078439364,0.8168068910256411,0.569960644038824
10,0.8578380326818328,0.4138376099564486,0.8157051282051282,0.5825908646369592
11,0.8576578019865427,0.40703976064567116,0.8023838141025641,0.6478387875816761
12,0.8577779557834028,0.4045132199795878,0.8293269230769231,0.5268991558979719
13,0.862483979493752,0.3937728417985373,0.8580729166666666,0.43161654902192265
14,0.8651674142902915,0.3908265250991912,0.8051883012820513,0.6256343785386819
15,0.8663088753604614,0.3804098418682518,0.8497596153846154,0.44730619427103263
16,0.8691324895866709,0.3765718101958593,0.8531650641025641,0.4531276652064079
17,0.8681111823133612,0.37401725701780053,0.83984375,0.49404423425977045
18,0.8706143864146108,0.36838437114072053,0.8127003205128205,0.6134260868032774
19,0.87487984620314,0.3588835015983698,0.8675881410256411,0.39528671929087394
20,0.8769224607497597,0.3555311407322152,0.8435496794871795,0.48546443573939496
21,0.8756608458827299,0.35530852686610215,0.8421474358974359,0.5073332617489191
22,0.8800016005121639,0.34906400301316026,0.8456530448717948,0.48053889950880635
23,0.8790691148171905,0.3490593365517238,0.8486578525641025,0.4672662219366966
**24,0.8826097404677988,0.3376938025850548,0.8743990384615384,0.38871548630488223**
25,0.8821090996475489,0.3360536029345712,0.8433493589743589,0.49158675987751055
26,0.8856336110221082,0.33037694730043643,0.8685897435897436,0.4002963486008155
27,0.8871155078500481,0.3245861655395408,0.8494591346153846,0.49426455604724395
28,0.8870554309516181,0.32317237779426483,0.8536658653846154,0.47174655741606003
29,0.8867550464594681,0.32392090227179177,0.8385416666666666,0.49987101822327346
30,0.8890379685998078,0.3214647053564561,0.8475560897435898,0.4839865692341939
31,0.8896988144825376,0.31493389705785074,0.8500600961538461,0.45689149162708187
32,0.8924223005446972,0.3106276677400399,0.8586738782051282,0.4500655401020478
33,0.8909404037167574,0.3076592045921196,0.8284254807692307,0.5773284009251839
**34,0.8954461710990067,0.2996342634971384,0.8802083333333334,0.37583819251412**
35,0.8936238385132971,0.3028916935652771,0.8655849358974359,0.43110579897004825
36,0.8950456584428068,0.30244991495293405,0.8631810897435898,0.43544238557418186
37,0.8962467989756722,0.2964690421611338,0.8727964743589743,0.4047875019411246

In [20]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 22s 2ms/step
Test loss: 0.4042066404819489
Test accuracy: 0.873


In [21]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


### Loading the saved weights to evaluate

In [22]:
from keras.models import load_model
model = load_model("weights.best.hdf5")
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 25s 2ms/step
Test loss: 0.3754211556315422
Test accuracy: 0.8803
